In [133]:
import yfinance as yf
from math import sqrt
import pandas as pd
import numpy as np 

In [134]:
# List of Columns
list_columns = ['Cumulative Returns', 'Max Drawdown', 'Sharpe Ratio', 'Sortino Ratio', 'Volatility']

In [135]:
class Index:
    
    list_of_indices = []
    index_tickers = []

    
    def __init__(self, ticker, risk_free_rate):
        self.ticker = ticker
        self.risk_free_rate = risk_free_rate
        self.start_date = "2010-01-01"
        self.end_date = "2023-05-01"
        self.period = '1d'
        self.data = self.fetch_data()
        
        # OLHC DATA: 
        self.open = self.data['Open']
        self.close = self.data['Close']
        self.high = self.data['High']
        self.low = self.data['Low']
        
        # Creating list of indices using instances: 
        Index.list_of_indices.append(self)
        
        # Creating list of indices tickers using instances: 
        Index.index_tickers.append(self.ticker)
        
    def fetch_data(self):
        return yf.download(self.ticker, start=self.start_date, end=self.end_date, period=self.period, progress=False)
        
        

# Defining instance of Index for each international index: 
ftse100 = Index("^FTSE", 0.02)
nk225 = Index("^N225", 0.01)
sp500 = Index("^GSPC", 0.03)
djia = Index("^DJI", 0.03)
nifty = Index("^NSEI", 0.05)

In [136]:
class Equity:
    
    list_of_equities = []
    equities_tickers = []
    
    def __init__(self, ticker, risk_free_rate):
        self.ticker = ticker
        self.risk_free_rate = risk_free_rate
        self.start_date = "2010-01-01"
        self.end_date = "2023-05-01"
        self.period = '1d'
        self.data = self.fetch_data()
        
        # OLHC DATA: 
        self.open = self.data['Open']
        self.close = self.data['Close']
        self.high = self.data['High']
        self.low = self.data['Low']
        
        # Creating list of equities using instances: 
        Equity.list_of_equities.append(self)
        
        # Creating list of equities tickers using instances: 
        Equity.equities_tickers.append(self.ticker)
        
    def fetch_data(self):
        return yf.download(self.ticker, start=self.start_date, end=self.end_date, period=self.period, progress=False)

# Defining instance of Equity for each international equity:
apple = Equity("AAPL", 0.02)
amazon = Equity("AMZN", 0.03)
nestle = Equity("NSRGF", 0.01)
reliance = Equity("RELIANCE.NS", 0.05)
toyota = Equity("TM", 0.01)

In [137]:
Equity.list_of_equities

In [138]:
close_prices = ftse100.close
open_prices = ftse100.open

# Daily Returns and Cumulative Returns: 
Price Returns can be calculated for various time periods, such as daily, monthly, quarterly, or annually, but in our case it is daily, so the formula to calculate returns would be as follows:

    `returns = (Current Value - Initial Value) / Initial Value`

Where:
- `Current Value` is the value of the investment at the end of the period.
- `Initial Value` is the value of the investment at the beginning of the period.


Assumptions:

1. The stock is bought at the opening price of each trading day.
2. The stock is sold at the closing price of each trading day.
3. There are no transaction costs or fees involved.
4. Dividends or other income generated by the stock are not considered.

In [139]:
def dailyReturns(asset):
    return (asset.close - asset.open)/ asset.open 


def cumulativeReturns(asset):
    
    # Calculating daily returns, for a trading day: 
    dailyReturn = dailyReturns(asset)
    
    # Calculate the cumulative returns by multiplication: 
    cumulativeReturn = ((1 + dailyReturn).prod() - 1)
    
    return cumulativeReturn

In [140]:
dailyReturns(apple)

Date
2010-01-04    0.002718
2010-01-05   -0.001025
2010-01-06   -0.015906
2010-01-07   -0.005525
2010-01-08    0.007989
                ...   
2023-04-24    0.002000
2023-04-25   -0.008596
2023-04-26    0.004293
2023-04-27    0.019493
2023-04-28    0.007063
Length: 3353, dtype: float64

In [141]:
cumulativeReturns(ftse100)

0.44022202939109056

# Volatility: 
Volatility describes changes over a specific period of time you simply take the standard deviation and multiply that by the square root of the number of periods in question:

``` vol = σ√T ```

where:

- v = volatility over some interval of time
- σ =standard deviation of returns
- T = number of periods in the time horizon

In [142]:
def Volatility(asset, adjust = True):
    
    # Define number of trading days:
    T = asset.data.shape[0]
    
    # Standard Deviation of daily returns:
    std = dailyReturns(asset).std()
    
    # Implementing adjusted volatility formula:
    if(adjust): 
        volatility = std*sqrt(T)
    else: 
        volatility = std
        
    return volatility

In [143]:
Volatility(ftse100)

0.5937192342293881

# Sharpe Ratio: 

In [144]:
# # Define the risk-free rate (e.g., Treasury bill yield)
# risk_free_rate = 0.01

def excessReturns(asset):
    return dailyReturns(asset) - asset.risk_free_rate/252

def sharpeRatio(asset):
    avg_xr = np.mean(excessReturns(asset))
    std_xr = np.std(excessReturns(asset))
    sharpe = avg_xr/std_xr
    return sharpe

In [145]:
sharpeRatio(apple)

0.020567460594453284

In [146]:
dailyReturns(apple)

Date
2010-01-04    0.002718
2010-01-05   -0.001025
2010-01-06   -0.015906
2010-01-07   -0.005525
2010-01-08    0.007989
                ...   
2023-04-24    0.002000
2023-04-25   -0.008596
2023-04-26    0.004293
2023-04-27    0.019493
2023-04-28    0.007063
Length: 3353, dtype: float64

# Sortino Ratio:

In [147]:
def sortinoRatio(asset):
    
    avg_dr = dailyReturns(asset).mean()

    # Calculate the downside deviation
    negative_daily_returns = dailyReturns(asset)[dailyReturns(asset) < 0]

    # downside_deviation 
    std_nr = negative_daily_returns.std()

    sortino = (avg_dr - asset.risk_free_rate/252) / std_nr
    
    return sortino

In [148]:
sortinoRatio(apple)

0.030324491155201642

# Maximum Drawdown: 

In [149]:
def maxDrawdown(asset):
    peak = asset.close[0]
    drawdown = 0.0 
    
    for price in asset.close:
        if price > peak:
            peak = price
        else: 
            curr_drawdown = (price - peak)/peak
            if curr_drawdown < drawdown:
                drawdown = curr_drawdown
    mdd = drawdown
    
    return mdd

In [150]:
maxDrawdown(nk225)

-0.3179889650850191

## Final Dataframe: 

In [151]:
index_data_list = []
for index in Index.list_of_indices:
    
    index_data = {
            'Cumulative Returns': cumulativeReturns(index),
            'Max Drawdown': maxDrawdown(index),
            'Sharpe Ratio': sharpeRatio(index),
            'Sortino Ratio': sortinoRatio(index),
            'Volatility': Volatility(index)
        }
    index_data_list.append(index_data)

In [152]:
df_index = pd.DataFrame(index_data_list, index=Index.index_tickers)
df_index

,Cumulative Returns,Max Drawdown,Sharpe Ratio,Sortino Ratio,Volatility
^FTSE,0.440222,-0.366055,0.007982,0.010309,0.593719
^N225,-0.344667,-0.317989,-0.013963,-0.017607,0.521219
^GSPC,1.226744,-0.339250,0.017481,0.022078,0.542172
^DJI,1.342443,-0.370862,0.019345,0.024780,0.528546
^NSEI,-0.906675,-0.384399,-0.096339,-0.136963,0.523440


In [153]:
equity_data_list = []
for equity in Equity.list_of_equities:
    
    equity_data = {
            'Cumulative Returns': cumulativeReturns(equity),
            'Max Drawdown': maxDrawdown(equity),
            'Sharpe Ratio': sharpeRatio(equity),
            'Sortino Ratio': sortinoRatio(equity),
            'Volatility': Volatility(equity)
        }
    equity_data_list.append(equity_data)
    
df_equity = pd.DataFrame(equity_data_list, index=Equity.equities_tickers)
df_equity

,Cumulative Returns,Max Drawdown,Sharpe Ratio,Sortino Ratio,Volatility
AAPL,1.467242,-0.443769,0.020567,0.030324,0.811384
AMZN,0.148333,-0.561453,0.003478,0.005241,0.951763
NSRGF,-0.717969,-0.271262,-0.041993,-0.055640,0.519687
RELIANCE.NS,-0.955643,-0.450884,-0.063299,-0.099132,0.921211
TM,-0.428564,-0.383593,-0.025537,-0.035037,0.411225


# Saving to Markdown:

In [154]:
from tabulate import tabulate

# Tabulating both dataframes:
table1 = tabulate(df_index, headers='keys', tablefmt='pipe')
table2 = tabulate(df_equity, headers='keys', tablefmt='pipe')

# Concatenating the tables in markdown:
combined_tables = table1 + '\n\n' + table2

# writing to the markdown file:
with open('tables.md', 'w') as f:
    f.write(combined_tables)